**WARNING:** Please note that it is not good practice to store a pre-rendered notebook externally (e.g. in GitHub or GitLab). This has been provided for training purposes only.

----------------------

## **A (quick) Introduction to Python**

In this session, we will dive into some Exploratory Data Analysis using Python. Our 3 aims here will be -
- Reading in and exporting data
- Dataframes and cleaning data
- Summary statistics and basic manipulation

In [ ]:
%pwd     # present working directory

### Import and setup

Firstly, we need to import the Python packages that we will be using in this session:
- `pandas` (go-to tools for data analysis and manipulation)
- `numpy` (maths functions and working with arrays)
- `matplotlib` (baseline visualisation package)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

You will need to download 2 datasets for this session. The URLs can be found in the `url_links.txt` file in the `data` folder.

Unzip the broadband data and copy all data files into the `data` folder.

The following cell will rename the files and remove one file that is not required.

In [ ]:
broadband_name = "data/202209_fixed_oa_res_coverage_r02.csv"     # check that this is the name of your OFCOM file
new_broadband_name = "data/ofcom_coverage_202209.csv"

lookup_name = "data/OA21_RGN22_LU.csv"      # check that this is the name of your lookup file
new_lookup_name = "data/ons_region_lookup_2022.csv"

popest_name = "data/sape23dt10dmid2020coaunformattedsyoaestimatesnortheast.xlsx"      # check that this is the name of your lookup file
new_popest_name = "data/pop_estimates_neengland_2020.xlsx"

file_to_drop = "data/202209_fixed_oa_coverage_r02.csv"

if os.path.exists(file_to_drop):
    os.remove(file_to_drop)

if os.path.exists(broadband_name):
    os.rename(broadband_name, new_broadband_name)
    
if os.path.exists(lookup_name):
    os.rename(lookup_name, new_lookup_name)
    
if os.path.exists(popest_name):
    os.rename(popest_name, new_popest_name)

Pandas is a versatile library which is capable of importing and interpreting multiple file formats ranging from flat files, databases and even parquet files. Common **read** functions include -
- `pd.read_csv()`
- `pd.read_excel()`
- `pd.read_json()`
- ...

Note that some will require additional installations. In particular, Excel files can be a bit more involved because we need to consider which sheet(s) we require and handle any header rows etc.

In [ ]:
coverage = pd.read_csv("data/ofcom_coverage_202209.csv")    # can use a relative path
lookup = pd.read_csv("data/ons_region_lookup_2022.csv")
# lookup = pd.read_csv("data/OA21_RGN22_LU.csv", dtype={"rgn22nmw": "str"})


When importing an Excel file, we must ensure that we have `openpyxl` installed and/or upgraded to the latest verson i.e.   
`pip install openpyxl` *or*   
`pip install --upgrade openpyxl`

In [ ]:
# ensure openpyxl upgraded to latest version (3.1.2)
popest = pd.read_excel("data/pop_estimates_neengland_2020.xlsx", sheet_name="Mid-2020 Persons", skiprows=4)

### Data inspection

We can inspect our data in a number of ways:
- using `.head()` for the top rows (a number can be specified, default is 5)
- using `.tail()` as above, for bottom rows
- using `.sample()` as above, for random rows (a random seed can be used)
- index slicing
- using `.loc` to specify rows and columns by name
- using `.iloc` to specify rows and columns by index reference

In [ ]:
# top 2 rows of our `coverage` data
coverage.head(2)

Notice the ellipsis (...) masking some of the rows i.e. the dataframe is too big to display by default.

In [ ]:
pd.options.display.max_columns = None     # this setting will now persist
coverage.head(3)

We can also print the values in a dataframe:

In [ ]:
print(lookup.head(2))
print(lookup.tail(2))
print(lookup.sample(2))

Notice the **NaN** values in the `lookup` table. We can summarise the extent of these easily in Python:

In [ ]:
lookup.isna().sum()

These **NaN** values are easily explained: Welsh is not provided for regions outside of Wales. Although fairly arbitrary, we could handle these missing values in a number of ways:
- filling all gaps with a value
- dropping rows with containing missing values
- assign values based on another column (or even an external function)

In [ ]:
# filling all gaps with a value
lookup["rgn22nmw"].fillna("not in Wales").sample(5, random_state=9)

In [ ]:
# dropping rows with missing values (leaves only Welsh OAs, in this example)
lookup.dropna(subset=["rgn22nmw"])

In [ ]:
# applying a dictionary to the region names in English to provide the Welsh names
welsh_regions = {"London": "Llundain",
                 "North West": "Gogledd Orllewin Lloegr",
                 "Yorkshire and The Humber": "Swydd Efrog a'r Humber",
                 "North East": "Gogledd-ddwyrain Lloegr",
                 "West Midlands": "Gorllewin Canolbarth Lloegr",
                 "East Midlands": "Dwyrain Canolbarth Lloegr",
                 "South West": "De-orllewin Lloegr",
                 "East of England": "Dwyrain Lloegr",
                 "South East": "De-ddwyrain Lloegr",
                 "Wales": "Cymru"}

lookup["rgn22nmw"] = lookup["rgn22nm"].map(welsh_regions)
lookup.sample(5, random_state=19)

We'll apply a 'treatment' in a bonus example at the end of this notebook!!!

We can easily slice a dataframe if we know which columns (or rows) we want. We use [] to specify a column name or [[]] for multiple column names. You may also have seen columns being called using the (example) `lookup.oa21cd` notation. This is not considered best practice but it can become useful at times, for example for more complex visualisations.

In [ ]:
# rows 10-19 using index slicing
popest[9:19]

In [ ]:
# column slicing
lookup[["oa21cd", "rgn22nm"]]

# row and column slicing
# lookup[:5][["oa21cd", "rgn22nm"]]

Perhaps we decide that there are 2 columns we are interested in, in particular: **SFBB availability (% premises)** and **Gigabit availability (% premises)**. Earlier, we displayed the initial rows of the `coverage` dataframe i.e. `coverage.head()`. We can see that our 2 columns are column FOUR and column ELEVEN. Python uses zero-indexing (where the first column is referenced as column ZERO) so we will need to access column THREE and column TEN.

In [ ]:
# index location
coverage.iloc[:5, [3,10]]    # i.e. top 5 rows (:5) and a list of column indices

In [ ]:
# Alternatively, we can access these rows and columns by name:
coverage.loc[:5, ["SFBB availability (% premises)", "Gigabit availability (% premises)"]]

We can also investigate the data structure: columns, rows, data types.

In [ ]:
lookup.shape      # numer of rows and columns as a tuple
#len(lookup)     # number of rows only
#len(lookup.columns)     # number of columns only

In [ ]:
# elements of a tuple can be accessed by index
lookup.shape[0]

In [ ]:
# note the use of f-string to embed variable values within a string
print(f"In the lookup table, there are {lookup.shape[0]} rows and {lookup.shape[1]} columns")

In [ ]:
lookup.columns

In [ ]:
lookup.columns.tolist()    # functions and/or methods can often be chained

In [ ]:
coverage.dtypes
#coverage.info()

We can also filter our dataframe in terms of data type:

In [ ]:
coverage.select_dtypes(include=["object", "float64"]).head(2)
#coverage.select_dtypes(exclude=["int"])

We can generate summary statistics for the whole dataframe (numerical variables only) or specific columns. Here a negative skew in the distribution of Gigabit coverage by output area:

In [ ]:
coverage.describe()

In [ ]:
coverage["Gigabit availability (% premises)"].describe()

#coverage.describe()    # for ALL columns with numerical variables

### **Descriptive statistics**

**pandas** is a huge library of methods for us to interrogate diverse data types with flexibility.

#### *range and averages*

In [ ]:
col = "Gigabit availability (% premises)"
print(f"Minimum: {coverage[col].min()}")
print(f"Maximum: {coverage[col].max()}")
print(f"Median: {coverage[col].median()}")
print(f"Other quantiles: \n{coverage[col].quantile(q=[0.25, 0.75]).values}")
print(f"Mode: {coverage[col].mode()[0]}")

# coverage[col].quantile(q=np.arange(0, 1.1, 0.1))     # deciles


#### *spread*

In [ ]:
coverage[col].std()      # standard deviation
#coverage[col].var()     # variance

We will look at visualisations in a later session but it is helpful that `pandas` uses the `matplotlib` library under the hood. This allows for a direct and responsive approach to visualisation.

In [ ]:
coverage["Gigabit availability (% premises)"].plot(kind="hist");
#coverage["Gigabit availability (% premises)"].hist();

We can also interrogate columns for unique values and missing values.

In [ ]:
# unique variables
lookup["rgn22nm"].unique()

In [ ]:
# number of unique variables in a given column
lookup["rgn22nm"].nunique()

In [ ]:
# total missing values
lookup["rgn22nmw"].isna().sum()

### **Basic feature engineering**

As part of our work, we might like to look at the prevalence of properties - by output area - with the option of the fastest broadband speeds. We can use binning to create new variables. Two approaches might be useful here:    
- `pd.cut` divides values into bins of broadly equal range (or user-defined)   
- `pd.qcut` divides values into bins with broadly equal values assigned

In [ ]:
# 5 bins of equal range
coverage["fast_bin_pct_prems"] = pd.cut(coverage["% of premises with >=300Mbit/s download speed"], bins=5)
coverage["fast_bin_pct_prems"].value_counts()

Perhaps we also want to comment on where output areas sit in terms of numbers of premises with the option of gigabit connection.

In [ ]:
# we can also specify the bin ranges but note the 0 behaviour
coverage["gig_bin_prems"] = pd.cut(coverage["Number of premises with Gigabit availability"], bins=coverage["Number of premises with Gigabit availability"].quantile(q=[0,0.25,0.5,0.75,1]))
coverage["gig_bin_prems"].value_counts().sort_index()

In [ ]:
# and `qcut` distributes bin allocation equally
coverage["gig_qbin_prems"] = pd.qcut(coverage["Number of premises with Gigabit availability"], q=4, labels=["First quartile", "Second quartile", "Third quartile", "Fourth quartile"])
coverage["gig_qbin_prems"].value_counts().sort_index()

In [ ]:
# inspecting our new features
coverage[["% of premises with >=300Mbit/s download speed",
          "fast_bin_pct_prems",
          "Number of premises with Gigabit availability",
          "gig_bin_prems",
          "gig_qbin_prems"]]

Again, the focus of this session is not visualisation but `matplotlib` makes it easy for us to view the difference in bin allocation.

In [ ]:
plt.subplots(nrows=2, figsize=(20,10))
ax1 = plt.subplot(2,1,1)
coverage["fast_bin_pct_prems"].value_counts().sort_index().plot(kind="barh", xlabel="num_OAs")
ax2 = plt.subplot(2,1,2, sharex=ax1)
coverage["gig_qbin_prems"].value_counts().sort_index().plot(kind="barh", xlabel="num_OAs", color="orange");

### Data manipulation

We can filter our data in a number of ways. In the following examples, we will filter by specific conditions both directly and by defining a mask (i.e. binary labelling of each row in terms of it meeting specified condition(s).)

In [ ]:
# comparison operators can be used to define a condition
mask = popest["All Ages"]>=400      # ==, !=, >, >=, <, <=
mask.head(10)

In [ ]:
# direct filtering by condition
popest[popest["All Ages"]>=400]
#popest[mask]     # filtering using the mask defined earlier

We can also filter on a slice:

In [ ]:
popest.loc[:, "OA11CD"][mask]

### Multiple condition filtering

We might want to investigate all output areas in Wales with less than 20% of premises with a gigabit option.   
We can filter a dataframe by multiple conditions where each condition is encapsulated within () and divided by   
- **&** (and)  
- **|** (or)

In [ ]:
# as before, binary label for each row
mask = (coverage["Gigabit availability (% premises)"]<20) & (coverage["output_area"].str[0]=="W")
mask.value_counts()

In [ ]:
# viewing the rows meeting the conditions directly
coverage[(coverage["Gigabit availability (% premises)"]<20) & (coverage["output_area"].str[0]=="W")]
#coverage[mask]

If we wanted to repeat this for all output areas in North East England, we can filter these codes using the `.isin()` method.

In [ ]:
# create a list of OAs in North East and North West England
reqd_oas = lookup["oa21cd"][lookup["rgn22nm"].isin(["North East"])].tolist()

# define conditions, including one as being in the above list
low_gig_north_england = coverage[(coverage["Gigabit availability (% premises)"]<20) & (coverage["output_area"].isin(reqd_oas))]
low_gig_north_england

Next we might want to check that the superfast availability is adequate even though the above areas have fairly poor gigabit coverage. We can use the `sort_values()` method to present the output areas in order.

In [ ]:
low_gig_north_england.sort_values("SFBB availability (% premises)")     # ascending order is default

In [ ]:
low_gig_north_england["SFBB availability (% premises)"].hist(grid=False);

We might be interested in unpicking the age demographic in the output areas above with 25% superfast coverage or lower. Again, we will want to create a list of these affected OAs.

In our example, we will look at the prevalence of younger children. First of all, we create an aggregate a new feature to capture all ages from 0 to 11 in our `popest` dataframe.

In [ ]:
# popest.columns = popest.columns.astype("str")
popest["11_or_under"] = popest.loc[:, [0,1,2,3,4,5,6,7,8,9,10,11]].sum(axis=1)
popest.head()

In [ ]:
vuln_oas = low_gig_north_england["output_area"][low_gig_north_england["SFBB availability (% premises)"]<=25].tolist()

# preview the first 10 items in the list
vuln_oas[0:10]

We arrive at a filtered view of our original dataframe - North East England OAs with <20% gigabit and <=25% superfast - ordered by numbers of primary-age pupils.

In [ ]:
# filtering and sorting
ordered_popest = popest[["OA11CD", "LSOA11CD", "All Ages", "11_or_under"]][popest["OA11CD"].isin(vuln_oas)].sort_values("11_or_under", ascending=False)
ordered_popest.head(10)

In [ ]:
ordered_popest["pct_11_or_under"] = ordered_popest["11_or_under"]/ordered_popest["All Ages"]*100
ordered_popest.sort_values("pct_11_or_under", ascending=False).head(10)

Alternatively, we might want to list ALL the output areas by population estimate and by LSOA in turn:

In [ ]:
# note lists to provide parameters for multiple columns
popest.iloc[:, :3].sort_values(["LSOA11CD", "All Ages"], ascending=[True, False]).head(20)

Let's return to our UK broadband coverage data. We can use the `.map()` method apply a dictionary to an existing column to generate values in a new column. Here, let's take the initial (country) letter for each output area to create a column of country labels.

In [ ]:
coverage["country"] = coverage["output_area"].str[0].map({"E": "England",
                                                          "W": "Wales",
                                                          "S": "Scotland",
                                                          "N": "Northern Ireland"})

# this 'random' sample demonstrates the effect of the `.map()` method
coverage[["output_area", "country"]].sample(5, random_state=9)

In [ ]:
# columns can be renamed
coverage.rename(columns={"country": "uk_country"}, inplace=True)

# values can be replaced
coverage["uk_country"].replace("Northern Ireland", "N Ireland", inplace=True)

coverage["uk_country"].unique()

We can check that all 4 countries have been captured correctly:

In [ ]:
coverage["uk_country"].value_counts()
#coverage["output_area"].str[0].value_counts()

### **Deleting columns**

Deleting columns should only be done if absolutely necessary. However, particularly for larger datasets, it can be a useful way of freeing up memory. The following examples are provided for illustration only.

For this example, we decide that Next Generation Access (NGA) variables are no longer required. We will use the `.filter()` method to identify columns with the *NGA* acronym.

In [ ]:
coverage.filter(like="NGA").head(2)

In [ ]:
# using 'del' in Python 3
del coverage["% of premises with NGA"]      # permanent!
coverage.filter(like="NGA").head(2)

In [ ]:
# using the `.drop()` method
coverage.drop(columns=["Number of premises with NGA"], inplace=True)
coverage.filter(like="NGA").head(2)

### **Joining dataframes**  
**Note:** In this illustration, we are joining two dataframes with different output area codes (one Census 2011, the other Census 2021). Only those output area codes to feature in both dataframes will be included in an inner join. Consequently, we are using a left join in this illustration to ensure that all existing population estimates are retained, irrespective of output area code changes.

In [ ]:
# how many rows in each dataframe
len(coverage), len(lookup)

In [ ]:
cov_incl_region = pd.merge(coverage, lookup, left_on="output_area", right_on="oa21cd", how="left")
cir = cov_incl_region.copy()      # for use later on
cov_incl_region

In [ ]:
df = cov_incl_region.merge(popest, left_on="output_area", right_on="OA11CD", how="inner")     # inner join as we are only working with North-East England population estimates
df[0:3]

There are many methods available in Python to achieve the same (or similar) results. You will want to explore these:   
- `pd.merge`   
- `pd.concat`   
- `pd.join`   
- `pd.append`

We'll save this dataframe using the `pd.to_csv` method.

In [ ]:
df.to_csv("outputs/broadband_neengland_combined.csv")

### **Aggregations using the `.groupby()` method**

The `.groupby()` method allows us to slice a dataframe by any number of columns and aggregate the values in terms of any number of other columns.

So - in this first example, we aggregate the maximum % of premises unable to receive 2Mbit/s download speeds in each LSOA.

In [ ]:
df.groupby(["LSOA11CD"])["% of premises unable to receive 2Mbit/s"].agg("max").sort_values(ascending=False).head(10)

You can often leave out the `.agg()` method to achieve the same result.

In [ ]:
cov_incl_region.groupby(["rgn22nm"])["output_area"].count()

Or the slice and aggregations can be defined in the form of a dictionary passed to the `.agg()` method.

In [ ]:
# look at all the chained methods!
df.groupby("LSOA11CD").agg({"Number of premises unable to receive 2Mbit/s": np.max,
                            "Number of premises with >=300Mbit/s download speed":"median"}).sort_values("Number of premises unable to receive 2Mbit/s", ascending=False).reset_index().head(10)

### **A few other things worth knowing**

The `.copy()` method is useful for creating an independent copy of a dataframe. Without this, you create a view which meaning both will update concurrently.

In [ ]:
copy_df = cir.copy()     # this creates an INDEPENDENT copy
copy_df["rgn22nm"].replace({"North East": "NE England"}, inplace=True)
copy_count = len(copy_df[copy_df["rgn22nm"]=="NE England"])
orig_count = len(cir[cir["rgn22nm"]=="NE England"])
print(f"In the copy: {copy_count} rows matching")
print(f"In the original: {orig_count} rows matching")

In [ ]:
copy_df = cir      # this creates a DEPENDENT view
copy_df["rgn22nm"].replace({"North East": "NE England"}, inplace=True)
copy_count = len(copy_df[copy_df["rgn22nm"]=="NE England"])
orig_count = len(cir[cir["rgn22nm"]=="NE England"])
print(f"In the copy: {copy_count} rows matching")
print(f"In the original: {orig_count} rows matching")

This is so critical that `pandas` will return a Copy Warning recommending the use of the `.loc()` method instead:

In [ ]:
cov_incl_region.loc[cov_incl_region["rgn22nm"]=="North West", "rgn22nm"] = "NW England"
cov_incl_region.loc[cov_incl_region["rgn22nm"]=="North East", "rgn22nm"] = "NE England"
cov_incl_region["rgn22nm"].unique()

String variables can easily be manipulated. Here are a few examples:

In [ ]:
# replacing <SPACE> with <UNDERSCORE>
cov_incl_region["rgn22nm"].str.replace(pat=" ", repl="_").unique()
#cov_incl_region["rgn22nm"].unique()

In [ ]:
# rewriting in lowercase
cov_incl_region["rgn22nm"].str.lower().unique()